In [10]:
!pip install xlsxwriter

   ---------------------------------------- 0.0/165.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/165.1 kB ? eta -:--:--
   -------------- ------------------------ 61.4/165.1 kB 656.4 kB/s eta 0:00:01
   ---------------------------------- ----- 143.4/165.1 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 165.1/165.1 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\dperez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
!pip install pandas
!pip install psycopg2
!pip install dotenv
import sys
import xlsxwriter
import pandas as pd
import sys
import psycopg2
import os
from dotenv import load_dotenv
from datetime import date

today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
# Conectar a PostgreSQL
try:
    cnx = psycopg2.connect(
        user=DB_USERNAME,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        database=DB_DATABASE
    )
    print("Conexión exitosa")
except psycopg2.Error as e:
    print("Ocurrió un error al conectar a la base de datos:", e)


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\dperez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\dperez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Ocurrió un error al conectar a la base de datos: connection to server at "127.0.0.1", port 3306 failed: received invalid response to SSL negotiation: J




[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\dperez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
